In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [0]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, you need to use the persistence path as the following:
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code, so that every time the environment (kernel) starts, just run the following code:
import sys
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
!pip install google.colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
!pip install surprise

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 11.8MB 9.2MB/s eta 0:00:011
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1638528 sha256=bd4bd9769663d57e7259f262b5d21ac342b06e3b134e468fba09758fca1ec9f2
  Stored in directory: /home/aistudio/.cache/pip/wheels/fe/8a/5d/bd58292eda7ef5dbcf9496799ea9f9ced482db787b2fc876c1
Successfully built scikit-surprise


In [22]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.prediction_algorithms import SVD, SVDpp 
from surprise.model_selection import KFold, cross_validate

In [24]:
df = pd.read_csv('data/data53696/ratings.csv')
df.head(10)

TypeError: __init__() got an unexpected keyword argument 'max_rows'

   userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580
5       1      112     3.5  1094785740
6       1      151     4.0  1094785734
7       1      223     4.0  1112485573
8       1      253     4.0  1112484940
9       1      260     4.0  1112484826

In [25]:
reader = Reader(line_format="user item rating timestamp",
               sep=',',
               skip_lines=1)

In [26]:
data = Dataset.load_from_file('data/data53696/ratings.csv', reader=reader)

In [27]:
type(data)

surprise.dataset.DatasetAutoFolds

In [28]:
train_set = data.build_full_trainset()
print(type(train_set))

<class 'surprise.trainset.Trainset'>


In [29]:
df.shape

(1048575, 4)

In [30]:
# 数据集中共有7120名用户、14026部电影，共计有1048575个评分
train_set.n_users, train_set.n_items, train_set.n_ratings

(7120, 14026, 1048575)

In [31]:
# 评分表的最小和最大评分
train_set.rating_scale

(1, 5)

In [32]:
df['rating'].min(), df['rating'].max()

(0.5, 5.0)

In [33]:
df.min(axis=0)

userId               1.0
movieId              1.0
rating               0.5
timestamp    825499934.0
dtype: float64

In [34]:
# 返回输入的用户id是否在train_set中，True代表存在
train_set.knows_user(0)

True

> **funkSVD算法**

In [35]:
# biased=False代表使用funkSVD算法
model = SVD(biased=False)

In [36]:
# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)

In [37]:
%%time
for trainset, testset in kf.split(data):
    # 在训练集上训练
    model.fit(trainset)
    # 为测试集生成预测
    predictions = model.test(testset)
    # 计算rmse
    accuracy.rmse(predictions, 
                  verbose=True) # 默认值为True，代表将打印计算值

RMSE: 0.8733
RMSE: 0.8713
RMSE: 0.8745
CPU times: user 2min 13s, sys: 498 ms, total: 2min 14s
Wall time: 2min 14s


In [38]:
trainset.knows_user(0)

True

In [39]:
%%time
pred = model.predict(uid='0', iid='1', r_ui=3.5, verbose=True)

user: 0          item: 1          r_ui = 3.50   est = 3.53   {'was_impossible': True, 'reason': 'User and item are unknown.'}
CPU times: user 432 µs, sys: 8 µs, total: 440 µs
Wall time: 260 µs


In [40]:
%%time
pred = model.predict(uid='1', iid='2', r_ui=3.5, verbose=True)

user: 1          item: 2          r_ui = 3.50   est = 3.70   {'was_impossible': False}
CPU times: user 191 µs, sys: 4 µs, total: 195 µs
Wall time: 143 µs


> **BiasSVD算法**

In [41]:
# biased=True代表使用BiasSVD算法,默认值为True
biassvd = SVD(biased=True)
biassvd

In [42]:
# 3折交叉验证实例
kf = KFold(n_splits=3)

In [43]:
%%time
for trainset, testset in kf.split(data):
    # 在训练集上训练
    biassvd.fit(trainset)
    # 给测试集生成预测
    predictions = biassvd.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8480
RMSE: 0.8453
RMSE: 0.8429
CPU times: user 2min 12s, sys: 553 ms, total: 2min 13s
Wall time: 2min 13s


In [44]:
pred = biassvd.predict(uid=1, iid=2, r_ui=3.5, verbose=True)

user: 1          item: 2          r_ui = 3.50   est = 3.53   {'was_impossible': False}


In [45]:
biassvd.predict(uid='1', iid='2', r_ui=3.5, verbose=True)

user: 1          item: 2          r_ui = 3.50   est = 3.37   {'was_impossible': False}


Prediction(uid='1', iid='2', r_ui=3.5, est=3.366448429972915, details={'was_impossible': False})

> **SVD++算法**

In [46]:
from surprise.model_selection.split import train_test_split
svdpp = SVDpp()
trainset,testset = train_test_split(data, 
                                    test_size=0.2,
                                    random_state=30)
type(trainset), type(testset)

(surprise.trainset.Trainset, list)

In [47]:
trainset.n_users

7120

In [48]:
testset[:2]

[('2511', '3814', 4.5), ('923', '3243', 3.0)]

In [ ]:
%%time
svdpp.fit(trainset)

In [ ]:
%%time
predictions = svdpp.test(testset)

In [ ]:
%%time
accuracy.rmse(predictions, verbose=True)

In [ ]:
pred = svdpp.predict(uid=1, iid=2, r_ui=3.5, verbose=True)

In [ ]:
svdpp.predict(uid='1', iid='2', r_ui=3.5, verbose=True)
